# Integrate Metadata and Data 

> This notebook will likely be broken into at least two

In [ ]:
#| default_exp core

<!-- #| hide
from nbdev.showdoc import * -->

In [ ]:
import os

import numpy as np
import pandas as pd

import plotly.express as px

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

    # !conda install openpyxl -y
    # ! conda install h5py -y
# ! conda install hilbertcurve -y

## Access genome records

Working with these records proved tricky. Ultimately I need the nucleotide data in a tensor, but after using tassel to save  the data (`ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023`) as a table (along with position list and taxa list) it's too big to easily load (>30Gb). As a work around to easily access specific genomes, I split the table into a separate file for the header and each genome so that these files can be read piecemeal. See the Readme below for more details.

In [ ]:
def read_txt(path):
    with open(path, 'r') as f:
        data = f.read()
    return(data)

def print_txt(path):
    print(read_txt(path = path))

In [ ]:
AGPv4_path = '../data/zma/panzea/genotypes/GBS/v27/'

In [ ]:
print_txt(path = AGPv4_path+'Readme')

Getting a script to split the table by line and rename all to the taxa name didn't work. It's possible that this is from sed, but it's not worth debugging. Instead I'm doing this manually which is what I should have done to begin with.

1. use split to produce the needed files
split -l ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table.txt

2. move those to their own folder
mv xz* GBSv27_publicSamples_imputedV5_AGPv4-181023_Table/

3. go over all files in the folder, pull out column one, swap out the : for another character and rename it.



This last point was completed with the following shell script.

In [ ]:
print_txt(path = AGPv4_path+'rename_all.sh')

#!/usr/bin/bash
files_path='./ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table/'
out_path='./ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table/'
#out_path='./out/'
files=$(ls "$files_path")

#echo $files

for file in $files
do
    echo $file
    taxa=$(awk '{print $1}' <<< cat "$files_path$file")
    taxa_sub=$(sed -r 's/[:]/__/g' <<< "$taxa")
    #echo $taxa_sub
    cp $files_path$file $out_path$taxa_sub
    #echo $taxa
done



With that done, and with the summary files from tassel (position and taxa), the genomes can be individually loaded as needed.

In [ ]:
# Other than listing the taxa this isn't expected to be of much use for our purposes.
AGPv4_taxa=pd.read_table(AGPv4_path+'ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_TaxaList.txt')
AGPv4_taxa.head()

,Taxa,LibraryPrepID,Status,DNAPlate,GENUS,INBREEDF,SPECIES,DNASample,Flowcell_Lane,NumLanes,...,GermplasmSet,Barcode,LibraryPlate,Tassel4SampleName,Population,LibraryPlateWell,SampleDNAWell,OwnerEmail,PEDIGREE,SeedLot
0,05-397:250007467,250007467,public,P3-GS-F,Zea,0.95,mays,05-397,C00R8ABXX_4,1.0,...,Margaret Smith lines,GTTGAA,P3-GS-F,05-397:C00R8ABXX:4:250007467,Inbred,F11,F11,esb33@cornell.edu,NaN,NaN
1,05-438:250007407,250007407,public,P3-GS-F,Zea,0.95,mays,05-438,C00R8ABXX_4,1.0,...,Margaret Smith lines,GTATT,P3-GS-F,05-438:C00R8ABXX:4:250007407,Inbred,B03,B03,esb33@cornell.edu,NaN,NaN
2,12E:250032344,250032344,public,Ames12,Zea,0.95,mays,12E,81FE8ABXX_4,1.0,...,Ames,GCTGTGGA,Ames12,12E:81FE8ABXX:4:250032344,inbred,H08,H08,esb33@cornell.edu,12E,NaN
3,207:250007202,250007202,public,P1-GS-F,Zea,0.95,mays,207,C00R8ABXX_2,1.0,...,Margaret Smith lines,TACAT,P1-GS-F,207:C00R8ABXX:2:250007202,Inbred,E12,E12,esb33@cornell.edu,NaN,NaN
4,22612:250007466,250007466,public,P3-GS-F,Zea,0.95,mays,22612,C00R8ABXX_4,1.0,...,Margaret Smith lines,GTACTT,P3-GS-F,22612:C00R8ABXX:4:250007466,Inbred,F10,F10,esb33@cornell.edu,NaN,NaN


In [ ]:
# Useful for converting between the physical location and site
AGPv4_site = pd.read_table(AGPv4_path+'ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_PositionList.txt')
AGPv4_site.head()

,Site,Name,Chromosome,Position
0,0,S1_6370,1,52399
1,1,S1_8210,1,54239
2,2,S1_8376,1,54405
3,3,S1_9889,1,55917
4,4,S1_9899,1,55927


Retrieving a genome by taxa name:

In [ ]:
# The genomes are in a folder with an identical name as their source table
table_directory = AGPv4_path+'ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table/'
# Note however that the naming is altered to not use ':'
os.listdir(table_directory)[0:3]

['05-397__250007467', '05-438__250007407', '12E__250032344']

In [ ]:
def taxa_to_filename(taxa = '05-397:250007467'): return(taxa.replace(':', '__'))

taxa_to_filename(taxa = '05-397:250007467')

'05-397__250007467'

In [ ]:
def get_AGPv4(taxa):
    with open(table_directory+taxa, 'r') as f:
        data = f.read()    
    data = data.split('\t')
    return(data)

get_AGPv4('05-397__250007467')[0:4]

['05-397:250007467', 'T', 'T', 'A']

In addition to returning a specific taxa, the table's headers can be retieved with "taxa".

In [ ]:
get_AGPv4(taxa = 'taxa')[0:4]

['Taxa', '52399', '54239', '54405']

Converting between site and chromosome/position requires the `AGPv4_site` dataframe. A given record contains the taxa as well as the nucleotides, so with that entry excluded the chromosome / position can be paired up.

In [ ]:
len(get_AGPv4(taxa = 'taxa')), AGPv4_site.shape

(943456, (943455, 4))

In [ ]:
ith_taxa = '05-397:250007467'
res = get_AGPv4(taxa_to_filename(taxa = ith_taxa))   # Retrieve record
temp = AGPv4_site.loc[:, ['Chromosome', 'Position']]  
temp[res[0]] = res[1:]                               # Add Col. with Nucleotides
temp.head()

,Chromosome,Position,05-397:250007467
0,1,52399,T
1,1,54239,T
2,1,54405,A
3,1,55917,N
4,1,55927,N


## Look at SNP coverage

In [ ]:
mask = (temp.Chromosome == 1)

temp_pos = temp.loc[mask, ['Position']]

In [ ]:
temp_pos['Shift'] = 0
temp_pos.loc[1: , ['Shift']] = np.array(temp_pos.Position)[:-1]
temp_pos['Diff'] = temp_pos['Position'] - temp_pos['Shift']

temp_pos.loc[0, 'Diff'] = None

In [ ]:
temp_pos

,Position,Shift,Diff
0,52399,0,NaN
1,54239,52399,1840.0
2,54405,54239,166.0
3,55917,54405,1512.0
4,55927,55917,10.0
...,...,...,...
147145,306971046,306910117,60929.0
147146,306971061,306971046,15.0
147147,306971063,306971061,2.0
147148,306971073,306971063,10.0


In [ ]:
# px.histogram(temp_pos, x = 'Diff')

## Demonstrate Hilbert Curve

In [ ]:
# demonstrating the hilbert curve
temp = np.linspace(1, 100, num= 50)
# px.scatter(x = temp, y = [0 for e in range(len(temp))], color = temp)
px.imshow(temp.reshape((1, temp.shape[0])))

In [ ]:
hilbert_curve = HilbertCurve(p = 10, # iterations i.e. hold 4^p positions
                             n = 2    # dimensions
                            )
distances = list(range(len(temp)))

points = hilbert_curve.points_from_distances(distances)
# px.line(pd.DataFrame(points, columns = ['i', 'j']), x = 'i', y = 'j')

In [ ]:
dim_0 = np.max(np.array(points)[:, 0])+1 # add 1 to account for 0 indexing
dim_1 = np.max(np.array(points)[:, 1])+1
temp_mat = np.zeros(shape = [dim_0, dim_1])
temp_mat[temp_mat == 0] = np.nan         #  empty values being used for visualization

for i in range(len(temp)):
#     print(i)
    temp_mat[points[i][0], points[i][1]] = temp[i]
    
# temp2 = pd.DataFrame(points, columns = ['i', 'j'])
# temp2['value'] = temp
# px.scatter(temp2, x = 'i', y = 'j', color = 'value')

px.imshow(temp_mat)

In [ ]:
# Data represented need not be continuous -- it need only have int positions
# a sequence or a sequence with gaps can be encoded
hilbert_curve = HilbertCurve(p = 10, # iterations i.e. hold 4^p positions
                             n = 2    # dimensions
                            )


fake_dists = list(range(len(temp)))
# Introdude a gap in the sequence
fake_dists = [e if e>10 else e+5 for e in fake_dists]
distances = fake_dists

points = hilbert_curve.points_from_distances(distances)
dim_0 = np.max(np.array(points)[:, 0])+1 # add 1 to account for 0 indexing
dim_1 = np.max(np.array(points)[:, 1])+1
temp_mat = np.zeros(shape = [dim_0, dim_1])
temp_mat[temp_mat == 0] = np.nan         #  empty values being used for visualization

for i in range(len(temp)):
#     print(i)
    temp_mat[points[i][0], points[i][1]] = temp[i]
px.imshow(temp_mat)


### Hilbert curve for one sequence

In [ ]:
temp_pos['Present'] = 1

In [ ]:
temp_pos.shape[0]

147150

4.0

In [ ]:
def calc_needed_hilbert_p(n_needed = 1048576,
                          max_p = 20):
    out = None
    for i in range(1, max_p):
        if 4**i > n_needed:
            out = i
            break
    return(out)

calc_needed_hilbert_p(n_needed=147150)

9

In [ ]:
temp_pos['Position']

0             52399
1             54239
2             54405
3             55917
4             55927
            ...    
147145    306971046
147146    306971061
147147    306971063
147148    306971073
147149    306971080
Name: Position, Length: 147150, dtype: int64

In [ ]:
# Data represented need not be continuous -- it need only have int positions
# a sequence or a sequence with gaps can be encoded
hilbert_curve = HilbertCurve(p = 10, # iterations i.e. hold 4^p positions
                             n = 2    # dimensions
                            )


fake_dists = list(range(len(temp)))
# Introdude a gap in the sequence
fake_dists = [e if e>10 else e+5 for e in fake_dists]
distances = fake_dists

points = hilbert_curve.points_from_distances(distances)
dim_0 = np.max(np.array(points)[:, 0])+1 # add 1 to account for 0 indexing
dim_1 = np.max(np.array(points)[:, 1])+1
temp_mat = np.zeros(shape = [dim_0, dim_1])
temp_mat[temp_mat == 0] = np.nan         #  empty values being used for visualization

for i in range(len(temp)):
#     print(i)
    temp_mat[points[i][0], points[i][1]] = temp[i]
px.imshow(temp_mat)


## Load phenotypic data to explore

For this I'm using data referenced in *Wallace et al 2014* which is available through panzea. This study refers to data from 9 studies (including itself) as a source of phenotypes for the NAM data. This combination of a large set of published GWAS hits, phenotypes, and many rils makes it ideal for use here. 

This file contains results I can use to check if my approaches are producing similar hits.

In [ ]:
Wallace_etal_2014_PLoSGenet_GWAS_hits = pd.read_table('../ext_data/zma/panzea/GWASResults/Wallace_etal_2014_PLoSGenet_GWAS_hits-150112.txt')

This file on I think matches *Buckler et al 2009*.

In [ ]:
temp = pd.read_excel('../ext_data/zma/panzea/GWASResults/JointGLMModels090324QTLLocations.xlsx', 
                     skiprows=1
                    ).rename(columns = {
    'Unnamed: 1': 'ASI', 
    'Unnamed: 2': 'Days to Anthesis', 
    'Unnamed: 3': 'Days to Silk'
})
temp.head()

/home/daniel/mambaforge/envs/dlgwas/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Heritability of line BLUPs across all crosses (excluding association panel):,ASI,Days to Anthesis,Days to Silk
0,Heritability of line BLUPs,0.775037,0.944619,0.942365
1,NaN,NaN,NaN,NaN
2,Heritability of line BLUPs within each cross:,NaN,NaN,NaN
3,B73×B97,0.64151,0.770513,0.766082
4,B73×CML103,0.613736,0.777381,0.734942


In [ ]:
# pull in some of the data that Wallace et al 2014 point to:

os.listdir('../ext_data/zma/panzea/phenotypes/')

['Brown_etal_2011_PLoSGenet_pheno_data-120523',
 'Brown_etal_2011_PLoSGenet_pheno_data-120523.zip',
 'Buckler_etal_2009_Science_flowering_time_data-090807',
 'Buckler_etal_2009_Science_flowering_time_data-090807.zip',
 'Coles_etal_2010_Genetics-Data_Supplement-091116',
 'Coles_etal_2010_Genetics-Data_Supplement-091116.zip',
 'Cook_etal_2012_kernel_comp_pheno_data-111202',
 'Cook_etal_2012_kernel_comp_pheno_data-111202.zip',
 'Hung_etal_2012_Heredity_data-110913',
 'Hung_etal_2012_Heredity_data-110913.zip',
 'Hung_etal_2012_PNAS_data-120523',
 'Hung_etal_2012_PNAS_data-120523.zip',
 'Kump_etal_2011_NatGen_SLB_pheno_data-120711',
 'Kump_etal_2011_NatGen_SLB_pheno_data-120711.zip',
 'NAM_2006_trait_herit-071008.xls',
 'NAM_2006_trait_means-071008.xls',
 'NAM_Flowering_Time_Summary_Population_Level-080409.xls',
 'Ogut_etal_2015_Heredity_DataSupplement-140604',
 'Ogut_etal_2015_Heredity_DataSupplement-140604.zip',
 'Peiffer2014Genetics_blupPhenos20150325(1).xlsx',
 'Peiffer2014Genetics_blup

In [ ]:
# with open('../ext_data/zma/panzea/GWASResults/Wallace_etal_2014_PLoSGenet_GWAS_hits-150112.txt', 'r') as f:
#     data = f.read()

In [ ]:
# data[0:200]

In [ ]:
# get an entry from AGPv4

agpv4_table_dir = '../data/zma/panzea/genotypes/GBS/v27/ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table/'

os.listdir(agpv4_table_dir)[0:2]

In [ ]:
%time
with open(agpv4_table_dir+'taxa', 'r') as f:
    data = f.read()


In [ ]:
%time
data = data.split('\t')
len(data)

In [ ]:
data[0:10]

In [ ]:
data[-10:-1]

In [ ]:
# data = pd.read_table(agpv4_table_dir+'taxa')

In [ ]:
data_int = [int(e) for e in data[1:]]

In [ ]:
!mamba install plotly -y

In [ ]:
import plotly.express as px


In [ ]:
px.histogram(x = data_int)

In [ ]:
import h5py

In [ ]:
agpv4 = h5py.File('../ext_data/zma/panzea/genotypes/GBS/v27/ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023.h5', 'r')

```
.
│
├── 
└──




├── Genotypes      (17282)
│   ├──05-397:250007467
│   ├──05-438:250007407
├── Positions      (6)
│   ├──AncestralAlleles
│   ├──ChromosomeIndices
│   ├──Chromosomes
│   ├──Positions
│   ├──ReferenceAlleles
│   ├──SnpIds

├── Taxa           (17281)
│   ├──05-397:250007467
│   ├──05-438:250007407
└── __DATA_TYPES__ (2)
    ├──Enum_Boolean
    ├──String_VariableLength
  
```

In [ ]:
keys_lvl1 = list(agpv4.keys())
keys_lvl1

In [ ]:
# e = ['AncestralAlleles',
#  'ChromosomeIndices',
#  'Chromosomes',
#  'Positions',
#  'ReferenceAlleles',
#  'SnpIds'][0]


# len(list(agpv4['Positions'][e]))

In [ ]:

[list(agpv4[e].keys())[0:10] for e in keys_lvl1]


In [ ]:
list(agpv4['Genotypes'].keys())[0:10]

In [ ]:
list(agpv4['Positions'].keys())

In [ ]:
list(agpv4['Taxa'].keys())[0:10]

In [ ]:
agpv4.name

In [ ]:
agpv4['Positions']['SnpIds']

In [ ]:
agpv4.close()

In [ ]:
# need a different solution. This file is ~30gb.
# agpv4 = pd.read_table('../data/zma/panzea/genotypes/GBS/v27/ZeaGBSv27_publicSamples_imputedV5_AGPv4-181023_Table.txt')

In [ ]:
os.listdir('../ext_data/zma/')

In [ ]:
# with open('../ext_data/zma/ensemble/mart_export.txt', 'r') as f:
#     data = f.read()
# data.split('\n')[0:10]

In [ ]:
os.listdir('../ext_data/zma/e2p2_computed/')

In [ ]:
with open('../ext_data/zma/e2p2_computed/B73_pathways.txt', 'r') as f:
    data = f.read()

In [ ]:
    

pd.read_table('../ext_data/zma/e2p2_computed/B73_pathways.txt')

In [ ]:
os.listdir('../ext_data/zma/ensemble/')

In [ ]:
import pandas as pd

pd.read_csv('../ext_data/zma/ensemble/mart_export.txt')

In [ ]:
os.listdir('../ext_data/zma/plant_reactome/')

In [ ]:
pd.read_table('../ext_data/zma/plant_reactome/Ensembl2PlantReactome_All_Levels.txt')

In [ ]:
# from dlgwas.internal import *

# ensure_dir_path_exists('../data/zma/panzea/genotypes/GBS/v27/')





'../ext_data/zma/panzea/phenotypes/Buckler_etal_2009_Science_flowering_time_data-090807/'